<a href="https://colab.research.google.com/github/fahmida185/NN-Projects/blob/master/chembl_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
# load and plot dataset
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
print("Using Tensorflow backend.")
print("Reading ChEMB data.")
# load dataset
series = read_csv('/content/drive/My Drive/NN_Course/chembl.csv/data.filtered.csv', header=0, index_col=0, squeeze=True)

Using Tensorflow backend.
Reading ChEMB data.


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [117]:
df=series[['Smiles','AlogP']]
df.dropna()

                                                    Smiles  AlogP
1        CC(=O)Oc1cc(S(=O)(=O)Nc2n[nH]c(Nc3ccc(C)cc3)n2...   3.52
2        O=C1N[C@H](c2c[nH]c3cc(Br)ccc23)CN=C1c1c[nH]c2...   4.83
3                CCOC(=O)/C=C/C(=O)Nc1cccc([N+](=O)[O-])c1   1.65
4                           O=C1Oc2ccccc2/C1=C\c1ccc(O)cc1   2.85
5        COc1c(CO)ccc2oc(C(=O)Nc3ccc(-c4ccc(S(=O)(=O)N[...   4.55
...                                                    ...    ...
2750108    Clc1ccc(COc2cncc(-c3ccc(CN4CCCCC4)s3)c2)c(Cl)c1   6.68
2750111          O=C1c2ccccc2C(=O)c2[nH]c(-c3ccccc3Br)nc21   3.61
2750112                CCCN1CCC=C(c2cc(-c3ccc(C)cc3)no2)C1   4.15
2750116  CCNC(=O)c1cc2c(-c3cc(C(C)(C)O)ccc3Oc3c(C)cc(F)...   5.06
2750118  COc1cc2c(c(OC)n1)[C@]1(O)[C@H](O)[C@H](CN(C)C)...   2.78

[909690 rows x 2 columns]


In [111]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import keras.models as km
import keras.layers as kl
import shelve, re
# split data into train and test
# train data preprocessing
train=df['Smiles'].str.strip()
trainX=train.str.replace("(","") 
trainX=trainX.str.replace(")","") 
trainX=trainX.str.replace("[","") 
trainX=trainX.str.replace("]","") 
trainX=trainX.str.replace("/","") 
trainX=trainX.str.replace("\\","")
trainX=trainX.str.replace("+","")
trainX=trainX.str.replace("@","")
trainX=trainX.str.replace("=","")
trainX=trainX.str.replace("-","")
trainX=trainX.str.replace("#","")
trainX=trainX.str.lower()
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler,StandardScaler

train_size = int(len(trainX) * 0.10)
test_size = len(trainX) - train_size

# binary encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(trainX)
train1= integer_encoded
train1= train1.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(train1)
scaled_X = scaler.transform(train1)

train_X= scaled_X[0:train_size]
test_X =  scaled_X[train_size:len(scaled_X)]
# print(train_X.shape)
# print(test_X.shape)

(90969, 1)
(818721, 1)


In [113]:
# transform Y data 
test= df['AlogP'].values
test= test.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(test)
scaled_Y = scaler.transform(test)
trainY=scaled_Y

ytrain= trainY[0:train_size]
ytest =  trainY[train_size:len(trainY)]

print(ytrain.shape)
print(ytest.shape)

(90969, 1)
(818721, 1)


In [114]:
X, y = train_X, ytrain
X = X.reshape(X.shape[0],1,1)
Xtest= test_X.reshape(test_X.shape[0],1,1)
print(X.shape)
print(y.shape)
print(Xtest.shape)
print(ytest.shape)

(90969, 1, 1)
(90969, 1)
(818721, 1, 1)
(818721, 1)


In [118]:
print("Building network.")
model = km.Sequential()
model.add(kl.LSTM(20, activation='tanh', input_shape=(1, 1), return_sequences=True))
model.add(kl.Dense(units=20, activation='relu'))
model.add(kl.Dense(units=1, activation='linear'))
model.compile(optimizer='adam', loss='mse')
print("Training network.")
model.fit(X,y,epochs=20,validation_data=(Xtest,ytest))
vy = model.history.history['val_loss']
ty = model.history.history['loss']
print("The training loss is ",ty[-1])
print("The test loss is ",vy[-1])

Building network.
Training network.
Epoch 1/20
2843/2843 [==============================] - 30s 10ms/step - loss: 0.0115 - val_loss: 0.0108
Epoch 2/20
2843/2843 [==============================] - 32s 11ms/step - loss: 0.0114 - val_loss: 0.0108
Epoch 3/20
2843/2843 [==============================] - 32s 11ms/step - loss: 0.0114 - val_loss: 0.0107
Epoch 4/20
2843/2843 [==============================] - 30s 10ms/step - loss: 0.0113 - val_loss: 0.0107
Epoch 5/20
2843/2843 [==============================] - 29s 10ms/step - loss: 0.0113 - val_loss: 0.0107
Epoch 6/20
2843/2843 [==============================] - 28s 10ms/step - loss: 0.0113 - val_loss: 0.0107
Epoch 7/20
2843/2843 [==============================] - 28s 10ms/step - loss: 0.0113 - val_loss: 0.0106
Epoch 8/20
2843/2843 [==============================] - 29s 10ms/step - loss: 0.0113 - val_loss: 0.0106
Epoch 9/20
2843/2843 [==============================] - 29s 10ms/step - loss: 0.0113 - val_loss: 0.0107
Epoch 10/20
2843/2843 [=====